# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_DFT(lattice, atoms, positions;
                  functionals=LDA(), symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770442452842                   -0.53    9.0    184ms
  2   -2.771682896110       -2.91       -1.31    1.0   95.2ms
  3   -2.771713778999       -4.51       -2.60    1.0    562ms
  4   -2.771714711561       -6.03       -3.95    2.0    133ms
  5   -2.771714714969       -8.47       -4.30    2.0    148ms
  6   -2.771714715241       -9.56       -5.42    1.0    131ms
  7   -2.771714715250      -11.09       -6.05    2.0    113ms
  8   -2.771714715250      -13.13       -6.86    1.0    100ms
  9   -2.771714715250   +  -14.45       -7.32    2.0    113ms
 10   -2.771714715250   +  -14.07       -7.71    1.0    115ms
 11   -2.771714715250      -14.07       -8.71    1.0    105ms


-0.00013457360780342912

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770493653354                   -0.52    9.0    225ms
  2   -2.771779263004       -2.89       -1.32    1.0    164ms
  3   -2.771801676749       -4.65       -2.45    1.0   96.2ms
  4   -2.771802004480       -6.48       -3.16    1.0   96.8ms
  5   -2.771802074304       -7.16       -4.22    2.0    529ms
  6   -2.771802074393      -10.05       -4.47    1.0   97.7ms
  7   -2.771802074474      -10.09       -5.77    1.0    100ms
  8   -2.771802074476      -11.73       -6.14    2.0    129ms
  9   -2.771802074476      -12.99       -6.72    1.0    101ms
 10   -2.771802074476      -14.31       -7.12    1.0    103ms
 11   -2.771802074476      -15.35       -7.47    1.0    104ms
 12   -2.771802074476   +  -14.51       -7.86    1.0    108ms
 13   -2.771802074476   +  -14.07       -8.33    2.0    128ms


0.017612222041997357

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457360780342912
Displaced dipole:  0.017612222041997357
Polarizability :   1.7746795649800784


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920945042e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766553118911e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852754712934e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694607765558e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088788915465e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.314053086458e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.657066359300e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.657066359300e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 9.888366069698e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.055178746925e-10
[ Info: GMRES linsolve in iter 2; step 3: normres = 1.518152137052e-12
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 1.518152137052e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 1.5181422435258237e-12
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.